<a href="https://colab.research.google.com/github/atacankavdir/academic/blob/main/volatility_baseline_modelrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
pip install numpy pandas tensorflow arch keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 6.2 MB/s eta 0:00:00


In [ ]:
import os
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Daily

In [ ]:
# Example usage:
# Load your data into a pandas DataFrame
eth_data = pd.read_csv('/content/drive/My Drive/DS Masters/Data/Ethereum Historical Data.csv', index_col='Date', parse_dates=True)

In [ ]:
eth_data.Price = eth_data.Price.str.replace(',', '').astype(float)

In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

# Calculate returns from the price data
eth_data['returns'] = eth_data['Close'].pct_change().dropna()

# Data Preparation for AP-GARCH and DFFNN
returns = eth_data['returns'].values[1:]  # remove the first NA value


# Step 1: Fit AP-GARCH Model
def fit_apgarch(returns):
    apgarch_model = arch_model(returns, vol='Garch', p=1, q=1, o=1, power=1.0)
    res = apgarch_model.fit(update_freq=5, disp='off')
    forecast = res.conditional_volatility
    return forecast

# Step 2: Prepare data for DFFNN
def prepare_data_for_dffnn(returns, forecasted_volatility):
    scaler = MinMaxScaler()
    scaled_returns = scaler.fit_transform(returns.reshape(-1, 1))
    scaled_volatility = scaler.transform(forecasted_volatility.reshape(-1, 1))

    # Create a feature set with returns and forecasted volatility
    features = np.hstack((scaled_returns[:-1], scaled_volatility[:-1]))  # shift by one to predict next step
    labels = scaled_returns[1:]

    return features, labels

# Step 3: Define DFFNN
def build_dffnn(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Main pipeline
forecasted_volatility = fit_apgarch(returns)
features, labels = prepare_data_for_dffnn(returns, forecasted_volatility)
model = build_dffnn(features.shape[1])

# Split data into training and testing sets (80-20 split for example)
split = int(len(features) * 0.8)
X_train, X_test = features[:split], features[split:]
y_train, y_test = labels[:split], labels[split:]

# Step 4: Train DFFNN
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

# Evaluation
mse = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {mse}")

# For a complete AP-DFFN model, you might iterate through the process, tweaking model parameters based on performance


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0006593. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


Epoch 1/50
8/8 [==============================] - 2s 46ms/step - loss: 0.3379 - val_loss: 0.2176
Epoch 2/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1513 - val_loss: 0.0971
Epoch 3/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0597 - val_loss: 0.0389
Epoch 4/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0210 - val_loss: 0.0199
Epoch 5/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0117 - val_loss: 0.0192
Epoch 6/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0126 - val_loss: 0.0188
Epoch 7/50
8/8 [==============================] - 0s 13ms/step - loss: 0.0114 - val_loss: 0.0179
Epoch 8/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0108 - val_loss: 0.0178
Epoch 9/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0108 - val_loss: 0.0176
Epoch 10/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0106 - val_loss: 0.0174
Epoch 11/50
8/8 [================

In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Assuming eth_data is your DataFrame with hourly Ethereum price data
eth_data['returns'] = eth_data['Close'].pct_change().dropna()
returns = eth_data['returns'].dropna().values

# Helper function to split data
def train_test_split(data, test_size=0.2):
    split_idx = int(len(data) * (1 - test_size))
    return data[:split_idx], data[split_idx:]

# 1. GARCH Model
def evaluate_garch(returns):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    res = model.fit(disp='off')
    forecast = res.conditional_volatility
    y_true, y_pred = train_test_split(returns**2)  # Squaring returns as GARCH models the variance
    mse = mean_squared_error(y_true[len(y_true) - len(y_pred):], y_pred**2)  # Squaring predictions to compare with true variance
    print(f'GARCH Model MSE: {mse}')
    return mse

# 2. AP-GARCH Model
def evaluate_apgarch(returns):
    model = arch_model(returns, p=1, q=1, o=1, vol='Garch', power=1.0)
    res = model.fit(disp='off')
    forecast = res.conditional_volatility
    y_true, y_pred = train_test_split(returns**2)
    mse = mean_squared_error(y_true[len(y_true) - len(y_pred):], y_pred**2)
    print(f'AP-GARCH Model MSE: {mse}')
    return mse

# 3. DFFNNs
def evaluate_dffnns(returns):
    scaler = MinMaxScaler()
    returns_scaled = scaler.fit_transform(returns.reshape(-1, 1)).flatten()
    X, y = returns_scaled[:-1], returns_scaled[1:]
    X_train, X_test = train_test_split(X)
    y_train, y_test = train_test_split(y)

    model = Sequential([
        Dense(64, activation='relu', input_shape=(1,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    predictions = model.predict(X_test).flatten()
    mse = mean_squared_error(y_test, predictions)
    print(f'DFFNNs MSE: {mse}')
    return mse

# 4. AP-DFFN
def evaluate_ap_dffn(returns):
    # Assuming AP-GARCH volatility forecasts and DFFNNs are combined here
    # This is a placeholder for your implementation based on the previous AP-DFFN guidelines
    # Step 1: Fit AP-GARCH Model
     def fit_apgarch(returns):
        apgarch_model = arch_model(returns, vol='Garch', p=1, q=1, o=1, power=1.0)
        res = apgarch_model.fit(update_freq=5, disp='off')
        forecast = res.conditional_volatility
        return forecast

     # Step 2: Prepare data for DFFNN
     def prepare_data_for_dffnn(returns, forecasted_volatility):
        scaler = MinMaxScaler()
        scaled_returns = scaler.fit_transform(returns.reshape(-1, 1))
        scaled_volatility = scaler.transform(forecasted_volatility.reshape(-1, 1))

        # Create a feature set with returns and forecasted volatility
        features = np.hstack((scaled_returns[:-1], scaled_volatility[:-1]))  # shift by one to predict next step
        labels = scaled_returns[1:]

        return features, labels

     # Step 3: Define DFFNN
     def build_dffnn(input_shape):
        model = Sequential([
            Dense(64, activation='relu', input_shape=(input_shape,)),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

     # Main pipeline
     forecasted_volatility = fit_apgarch(returns)
     features, labels = prepare_data_for_dffnn(returns, forecasted_volatility)
     model = build_dffnn(features.shape[1])

     # Split data into training and testing sets (80-20 split for example)
     split = int(len(features) * 0.8)
     X_train, X_test = features[:split], features[split:]
     y_train, y_test = labels[:split], labels[split:]

     # Step 4: Train DFFNN
     model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

     # Evaluation
     mse = model.evaluate(X_test, y_test, verbose=0)
     print(f"Test MSE: {mse}")
     return mse

# Load your data
#eth_data = pd.read_csv('your_ethereum_hourly_price_data.csv')  # Make sure to adjust the path

# Evaluate Models
garch_mse = evaluate_garch(returns)
apgarch_mse = evaluate_apgarch(returns)
dffnns_mse = evaluate_dffnns(returns)
apdfnn_mse = evaluate_ap_dffn(returns)


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0006593. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0006593. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


GARCH Model MSE: 3.050549452698324e-06
AP-GARCH Model MSE: 3.050549452698324e-06
3/3 [==============================] - 0s 4ms/step
DFFNNs MSE: 0.02092170474338698
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0006593. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


8/8 [==============================] - 1s 43ms/step - loss: 0.2028 - val_loss: 0.1217
Epoch 2/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0740 - val_loss: 0.0366
Epoch 3/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0172 - val_loss: 0.0187
Epoch 4/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0147 - val_loss: 0.0240
Epoch 5/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0156 - val_loss: 0.0195
Epoch 6/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0114 - val_loss: 0.0182
Epoch 7/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0114 - val_loss: 0.0187
Epoch 8/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0114 - val_loss: 0.0181
Epoch 9/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0109 - val_loss: 0.0178
Epoch 10/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0109 - val_loss: 0.0177
Epoch 11/50
8/8 [===========================

In [ ]:
#Log returns
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.metrics import mean_squared_error

In [ ]:
|def calculate_log_returns(prices):
    """
    Calculate the daily log returns of a given series of prices.

    Parameters:
    prices (pd.Series): A pandas Series containing the daily closing prices of an asset.

    Returns:
    pd.Series: A pandas Series containing the daily log returns.
    """
    # Calculate daily log returns
    log_returns = np.log(prices / prices.shift(1))
    return log_returns.dropna()  # Remove any NaN values that result from the shift

 # Calculate log returns
log_returns_eth = calculate_log_returns(eth_data['Price'].sort_index())
print(log_returns_eth)

Date
2016-03-11    0.016878
2016-03-12    0.078045
2016-03-13    0.153930
2016-03-14   -0.186977
2016-03-15    0.043825
                ...   
2024-04-10    0.010834
2024-04-11   -0.012092
2024-04-12   -0.077961
2024-04-13   -0.075439
2024-04-14    0.016328
Name: Price, Length: 2957, dtype: float64


In [ ]:
log_returns_eth

Date
2016-03-11    0.016878
2016-03-12    0.078045
2016-03-13    0.153930
2016-03-14   -0.186977
2016-03-15    0.043825
                ...   
2024-04-10    0.010834
2024-04-11   -0.012092
2024-04-12   -0.077961
2024-04-13   -0.075439
2024-04-14    0.016328
Name: Price, Length: 2957, dtype: float64

In [ ]:
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.metrics import mean_squared_error

def calculate_log_returns(prices):
    """Calculate daily log returns from prices."""
    return np.log(prices / prices.shift(1)).dropna()

def fit_garch_model(log_returns, p_orders, q_orders):
    """Fit GARCH models and perform parameter search for best MSE."""
    best_mse = np.inf
    best_model = None
    best_order = None
    #log_returns = 10*log_returns
    for p in p_orders:
        for q in q_orders:
            model = arch_model(log_returns, vol='Garch', p=p, q=q, dist='Normal')
            res = model.fit(disp='off')
            # Calculate the predicted conditional volatility
            predicted_volatility = res.conditional_volatility
            # Calculate MSE comparing squared log returns to squared predicted volatilities
            mse = mean_squared_error(log_returns**2, predicted_volatility**2)
            if mse < best_mse:
                best_mse = mse
                best_model = res
                best_order = (p, q)

    return best_model, best_order, best_mse


log_returns = calculate_log_returns(eth_data['Price'].sort_index())
best_model, best_order, best_mse = fit_garch_model(log_returns, p_orders=[1, 2], q_orders=[1, 2])

print("Best Model Order (p, q):", best_order)
print("Best Model Summary:")
print(best_model.summary())
print("Best mse:", best_mse)


Best Model Order (p, q): (2, 2)
Best Model Summary:
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                  Price   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                4830.15
Distribution:                  Normal   AIC:                          -9648.30
Method:            Maximum Likelihood   BIC:                          -9612.35
                                        No. Observations:                 2957
Date:                Sun, Apr 14 2024   Df Residuals:                     2956
Time:                        16:55:33   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu    

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002728. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002728. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning:

In [ ]:
pip install statsmodels


In [ ]:
#HAR
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

def calculate_log_returns(prices):
    """Calculate daily log returns from prices."""
    return np.log(prices / prices.shift(1)).dropna()

def prepare_har_data(log_returns, daily_lag=1, weekly_lag=5, monthly_lag=22):
    """Prepare data for HAR model with specified lags."""
    df = pd.DataFrame(log_returns.values, columns=['log_return'])
    df['daily'] = df['log_return'].shift(daily_lag)
    df['weekly'] = df['log_return'].rolling(window=weekly_lag).mean().shift(1)
    df['monthly'] = df['log_return'].rolling(window=monthly_lag).mean().shift(1)
    df.dropna(inplace=True)  # Drop rows with NaN values
    return df

def har_model(df):
    """Fit HAR model and return model and predictions."""
    X = df[['daily', 'weekly', 'monthly']]
    y = df['log_return'].dropna()
    X = sm.add_constant(X)  # adding a constant
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)
    return model, predictions

def parameter_search(prices, daily_range, weekly_range, monthly_range):
    """Search for best parameters based on MSE."""
    log_returns = calculate_log_returns(prices)
    #
    best_mse = float('inf')
    best_params = {}

    for d in daily_range:
        for w in weekly_range:
            for m in monthly_range:
                df = prepare_har_data(log_returns, daily_lag=d, weekly_lag=w, monthly_lag=m)
                _, predictions = har_model(df)
                mse = mean_squared_error(df['log_return'], predictions)
                if mse < best_mse:
                    best_mse = mse
                    best_params = {'daily': d, 'weekly': w, 'monthly': m, 'MSE': mse}

    return best_params

best_params = parameter_search(eth_data['Price'].sort_index(), daily_range=[1], weekly_range=[5], monthly_range=[22])
print(best_params)


{'daily': 1, 'weekly': 5, 'monthly': 22, 'MSE': 0.002674285431137921}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def calculate_log_returns(prices):
    """Calculate daily log returns from prices."""
    return np.log(prices / prices.shift(1)).dropna()

def build_model(n_layers, n_units, input_shape, dropout_rate=0.2, learning_rate=0.001):
    """Builds a Sequential neural network model."""
    model = Sequential()
    model.add(Dense(n_units, activation='relu', input_shape=(input_shape,)))
    for _ in range(n_layers - 1):
        model.add(Dense(n_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

def parameter_search(data, n_units_options, n_layers_options):
    """Searches for the best neural network architecture."""
    X = data.drop('log_return', axis=1)
    y = data['log_return']**2  # Predicting the squared returns as a proxy for volatility
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    best_mse = float('inf')
    best_params = None

    for n_units in n_units_options:
        for n_layers in n_layers_options:
            model = build_model(n_layers, n_units, input_shape=X_train.shape[1])
            model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
            predictions = model.predict(X_test)
            mse = mean_squared_error(y_test.fillna(0), predictions)
            if mse < best_mse:
                best_mse = mse
                best_params = {'n_layers': n_layers, 'n_units': n_units, 'MSE': mse}

    return best_params

# Example usage:
df['log_return'] = calculate_log_returns(eth_data['Price'].sort_index())

# Assuming we want to predict volatility from the squared returns
best_params = parameter_search(df, n_units_options=[10, 50], n_layers_options=[1, 2])
print("Best model parameters:", best_params)


1/1 [==============================] - 0s 111ms/step
Best model parameters: {'n_layers': 1, 'n_units': 50, 'MSE': 0.017908950058498414}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def calculate_log_returns(prices):
    """Calculate daily log returns from prices."""
    return np.log(prices / prices.shift(1)).dropna()

def build_dfnn(n_layers, n_units, input_shape, dropout_rate=0.2, learning_rate=0.001):
    """Builds a Deep Feedforward Neural Network (DFNN)."""
    model = Sequential()
    model.add(Dense(n_units, activation='relu', input_shape=(input_shape,)))
    for _ in range(n_layers - 1):
        model.add(Dense(n_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

def parameter_search(data, n_units_options, n_layers_options):
    """Searches for the best neural network architecture based on MSE."""
    X = data.drop('log_return', axis=1)
    y = data['log_return']**2  # Predicting squared returns as a proxy for volatility
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    best_mse = float('inf')
    best_params = None

    for n_units in n_units_options:
        for n_layers in n_layers_options:
            model = build_dfnn(n_layers, n_units, input_shape=X_train.shape[1])
            model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
            predictions = model.predict(X_test)
            mse = mean_squared_error(y_test.fillna(0), predictions)
            if mse < best_mse:
                best_mse = mse
                best_params = {'n_layers': n_layers, 'n_units': n_units, 'MSE': mse}

    return best_params

df['log_return'] = calculate_log_returns(eth_data['Price'].sort_index())

# Parameter search for optimal DFNN architecture
best_params = parameter_search(df, n_units_options=[10, 50], n_layers_options=[1, 3])
print("Best model parameters:", best_params)


1/1 [==============================] - 0s 66ms/step
Best model parameters: {'n_layers': 1, 'n_units': 50, 'MSE': 0.0006727218500663964}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def calculate_log_returns(prices):
    """Calculate daily log returns from prices."""
    return np.log(prices / prices.shift(1)).dropna()

def build_lstm(n_units, input_shape, dropout_rate=0.2, learning_rate=0.001):
    """Builds an LSTM network."""
    model = Sequential([
        LSTM(n_units, input_shape=input_shape, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(n_units),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

def parameter_search(data, n_units_options):
    """Searches for the best LSTM architecture based on MSE."""
    X = np.array(data.drop('log_return', axis=1))
    y = np.array(data['log_return']**2)  # Predicting squared returns as a proxy for volatility
    # Reshape X for LSTM [samples, time steps, features]
    X = X.reshape((X.shape[0], 1, X.shape[1]))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    best_mse = float('inf')
    best_params = None

    for n_units in n_units_options:
        model = build_lstm(n_units, input_shape=(X_train.shape[1], X_train.shape[2]))
        model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        if mse < best_mse:
            best_mse = mse
            best_params = {'n_units': n_units, 'MSE': mse}

    return best_params

# Example usage:
df['log_return'] = calculate_log_returns(eth_data['Price'].sort_index())

# Assuming we want to predict volatility from squared returns
best_params = parameter_search(df, n_units_options=[10, 20, 30, 40, 50, 100, 200, 500, 1000])
print("Best model parameters:", best_params)


1/1 [==============================] - 1s 1s/step
Best model parameters: {'n_units': 10, 'MSE': 5.443004211736995e-06}
